In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
import requests
import time
import asyncio
from telegram import Bot
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
from tabulate import tabulate
import sys
import numpy as np

In [2]:
topic_id = 1

In [3]:
caminho = os.path.join(r"C:\Users\55359\Documents\My Projects\pipeline-dados\Dados", "dados_games.jsonl")

In [4]:
df = pd.read_json(caminho, lines=True, dtype={"preco_anterior": str, "fracao_preco_anterior":str, "preco_atual":str, "fracao_preco_atual":str})

In [5]:
def limpar_numero(valor):
            if pd.isnull(valor):
                return valor
            return int(str(valor).replace('.', '').replace(',', ''))

In [6]:
colunas_substituir_none = ['preco_anterior', 'fracao_preco_anterior', 'preco_atual', 'fracao_preco_atual']
df[colunas_substituir_none] = df[colunas_substituir_none].replace(['None', None], 0)

In [7]:
df["preco_anterior"] = df["preco_anterior"].apply(limpar_numero)
df["fracao_preco_anterior"] = df["fracao_preco_anterior"].apply(limpar_numero)
df["preco_atual"] = df["preco_atual"].apply(limpar_numero)
df["fracao_preco_atual"] = df["fracao_preco_atual"].apply(limpar_numero)

df['preco_anterior'] = df["preco_anterior"].astype(str) + "," + df["fracao_preco_anterior"].astype(str)
df['preco_atual'] = df["preco_atual"].astype(str) + "," + df["fracao_preco_atual"].astype(str)

df = df.drop(columns=['fracao_preco_anterior', 'fracao_preco_atual'])

df['porcentagem_desconto'] = df["porcentagem_desconto"].str.extract('(\d+)')
df['porcentagem_desconto'] = df['porcentagem_desconto'].fillna(0)
df['porcentagem_desconto'] = df['porcentagem_desconto'].astype(int)

df = df.sort_values('porcentagem_desconto', ascending=True)
df = df.drop_duplicates(subset=['titulo'], keep='last')
df['data_coleta']  = datetime.now()
df['topico_de_envio'] = f"{topic_id}"

In [8]:
df_modificado = df

In [9]:
df_modificado['preco_anterior'] = df_modificado['preco_anterior'].str.replace(',', '.').astype(float)
df_modificado['preco_atual'] = df_modificado['preco_atual'].str.replace(',', '.').astype(float)
df_modificado['desconto_reais'] = df_modificado['preco_anterior'] - df_modificado['preco_atual']

In [10]:
def calcular_pontuacao(row, lojas_vistas):
    pontuacao = 0
    
    if row["porcentagem_desconto"] >= 50:
        pontuacao += 5
    elif row["porcentagem_desconto"] >= 45:
        pontuacao += 4
    elif row["porcentagem_desconto"] >= 40:
        pontuacao += 2
    elif row["porcentagem_desconto"] >= 35:
        pontuacao += 1
    
    if row["desconto_reais"] > 1000 and row["porcentagem_desconto"] < 50:
        pontuacao += 5
    elif row["desconto_reais"] > 600 and row["porcentagem_desconto"] < 40:
        pontuacao += 4
    elif row["desconto_reais"] > 300 and row["porcentagem_desconto"] < 20:
        pontuacao += 3
    elif row["desconto_reais"] > 100 and row["porcentagem_desconto"] < 10:
        pontuacao += 3
    
    if pd.notna(row["highlight"]):
        pontuacao += 1
    if pd.notna(row["vendido_por"]):
        pontuacao += 1
    if pd.notna(row["detalhe_envio"]) or pd.notna(row["detalhe_envio_2"]):
        pontuacao += 1
    
    if pd.notna(row["vendido_por"]) and row["vendido_por"] in lojas_vistas and row["vendido_por"] != "Por Mercado Livre":
        pontuacao -= 3
    
    if pd.notna(row["vendido_por"]):
        lojas_vistas.add(row["vendido_por"])

    if "Kit" in str(row["titulo"]):
        pontuacao += 2

    if "Jogo De" in str(row["titulo"]):
        pontuacao += 2

    return pontuacao

lojas_vistas = set()

df["pontuacao"] = df.apply(calcular_pontuacao, axis=1, lojas_vistas=lojas_vistas)

def classificar_relevancia(pontuacao):
    if pontuacao >= 8:
        return "Relevante"
    elif pontuacao >= 6:
        return "Alta-Relevância"
    elif pontuacao >= 5:
        return "Média-Relevância"
    elif pontuacao >= 2:
        return "Baixa-Relevância"
    else:
        return "Irrelevante"

df["relevancia"] = df["pontuacao"].apply(classificar_relevancia)

In [11]:
df_modificado['relevancia'].value_counts()

relevancia
Irrelevante         75
Baixa-Relevância    32
Média-Relevância     6
Alta-Relevância      4
Relevante            1
Name: count, dtype: int64

In [12]:
df_modificado.to_excel(r"C:\Users\55359\Downloads\base2.xlsx", index=False)